In [1]:
import logging
from typing import Any, Set

from automata.config.base import AgentConfigName
from automata.singletons.dependency_factory import dependency_factory
from automata.singletons.py_module_loader import py_module_loader
from automata.context_providers.symbol_synchronization import (
    SymbolProviderSynchronizationContext,
)
    
from automata.tools.base import Tool
from automata.tools.builders.context_oracle import ContextOracleToolkitBuilder

logger = logging.getLogger(__name__)

py_module_loader.initialize()

In [2]:
symbol_search = dependency_factory.get('symbol_search')
symbol_code_embedding_handler = dependency_factory.get('symbol_code_embedding_handler')
symbol_doc_embedding_handler = dependency_factory.get('symbol_doc_embedding_handler')
embedding_similarity_calculator = dependency_factory.get('embedding_similarity_calculator')

Using embedded DuckDB with persistence: data will be stored in: /Users/ocolegrove/AutomataNoCruft/automata/core/../../embedding_data/code_embedding
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction
Using embedded DuckDB with persistence: data will be stored in: /Users/ocolegrove/AutomataNoCruft/automata/core/../../embedding_data/doc_embedding_l2
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


In [3]:
context_oracle = ContextOracleToolkitBuilder(symbol_search, symbol_doc_embedding_handler, symbol_code_embedding_handler, embedding_similarity_calculator)

In [4]:
context = context_oracle._get_context("Provide new code which refactors the SymbolDocEmbeddingHandler class to be more robust.")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 622/622 [00:02<00:00, 269.83it/s]


In [5]:
print(context)



class SymbolDocEmbeddingHandler(SymbolEmbeddingHandler):
    'A class to handle the embedding of symbols'

    def __init__(self, embedding_db: VectorDatabaseProvider, embedding_builder: SymbolDocEmbeddingBuilder) -> None:
        super().__init__(embedding_db, embedding_builder)

    def process_embedding(self, symbol: Symbol) -> None:
        '\n        Process the embedding for a `Symbol` -\n        Currently we do nothing except update symbol commit hash and source code\n        if the symbol is already in the database.\n        '
        source_code = self.embedding_builder.fetch_embedding_source_code(symbol)
        if (not source_code):
            raise ValueError(f'Symbol {symbol} has no source code')
        if self.embedding_db.contains(symbol.dotpath):
            self.update_existing_embedding(source_code, symbol)
            return
        if (symbol.symbol_kind_by_suffix() == SymbolDescriptor.PyKind.Class):
            symbol_embedding = self.embedding_builder.build(so